# Statistical Testing

Kiểm tra dữ liệu giá tài sản có xu hướng quay về mức trung bình sau các biến động ngắn hạn hay không (tính dừng)

## Import module

In [ ]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd   
import numpy as np

## 1. Augmented Dickey-Fuller Test

**Augmented Dickey-Fuller (ADF) Test** là một phương pháp thống kê được sử dụng để kiểm tra tính dừng (stationarity) của một chuỗi thời gian.
- Một chuỗi thời gian được coi là dừng nếu các thuộc tính thống kê của nó (như trung bình, phương sai) không thay đổi theo thời gian. Điều này có nghĩa là các giá trị trong chuỗi không có xu hướng tăng hoặc giảm theo thời gian. Nếu một chuỗi là mean-reverting, nó thường có tính dừng.
- Các bước kiểm định ADF:
    - **Tính toán giá trị thống kê ADF (ADF Statistic)**: Đo lường sự thay đổi của chuỗi thời gian dựa trên độ lệch của các giá trị trước đó.
    - **Xác định p-value**
    - **Kết quả**:
        - Nếu p-value < 𝛼 (0.05), ta sẽ bác bỏ giả thuyết H0 (chuỗi có tính dừng).
        - Nếu p-value >  𝛼 (0.05) , không thể bác bỏ giả thuyết H0 (chuỗi có thể không dừng).

In [ ]:
df_ohc = pd.read_csv('/content/openhighclose_ETHUSDT_7d.csv')
df_ohc['start_at'] = pd.to_datetime(df_ohc['start_at'], unit='s')
df_stat = df_ohc.copy()

# Bước 1: Tính toán log returns từ cột close
df_stat['log_return'] = np.log(df_stat['close'] / df_stat['close'].shift(1))

# Áp dụng trung bình trượt 3 kỳ để làm mịn log returns
df_stat['log_return'] = df_stat['log_return'].rolling(window=3, center=True).mean()
df_stat = df_stat.dropna(subset=['log_return'])

In [ ]:
#Kiểm tra tính dừng

result = adfuller(df_stat['log_return'].dropna())

print(f'ADF Statistic: {result[0]}')
print(f'p_value: {result[1]}')

ADF Statistic: -15.528754953255493
p_value: 2.2345676828495575e-28


p_value < 0.05 => dữ liệu có tính dừng. Nó là chuỗi không tăng ko giảm, có thể áp dụng O-U model

## 2. Hurst Exponent

Hurst Exponent (H) giúp xác định đặc tính của chuỗi thời gian:
- Nếu 𝐻 ≈ 0.5, chuỗi là ngẫu nhiên (Brownian motion)
- Nếu H < 0.5, chuỗi có xu hướng mean reversion
- Nếu H > 0.5, chuỗi có xu hướng tự khuếch đại (trending).

In [ ]:
# Loại bỏ giá trị trùng lặp
df_stat['log_return'] = df_stat['log_return'].drop_duplicates()

# Loại bỏ giá trị bằng 0 hoặc âm (nếu có)
df_stat['log_return'] = df_stat['log_return'][df_stat['log_return'] > 0]

df_stat['close_smooth'] = df_stat['log_return'].rolling(window=3).mean()
df_stat['close_shifted'] = df_stat['log_return'] + 1e-5  # Dịch lên 0.00001

In [ ]:
def hurst_exponent(ts, max_lag=20):
    lags = range(2, max_lag)
    tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]

    # Kiểm tra giá trị của tau
    if any(t == 0 for t in tau):
        print("Có giá trị 0 trong tau, không thể lấy log.")
        return np.nan

    reg = np.polyfit(np.log(lags), np.log(tau), 1)
    return reg[0] * 2.0


hurst = hurst_exponent(df_stat["close_shifted"].dropna())
print(f'Hurst Exponent: {hurst}')

if hurst < 0.5:
    print('Chuỗi có xu hướng mean-reverting')
elif hurst == 0.5:
    print('Chuỗi có tính ngẫu nhiên')
else:
    print("Chuỗi có xu hướng bền vững")

Có giá trị 0 trong tau, không thể lấy log.
Hurst Exponent: nan
Chuỗi có xu hướng bền vững


**Nguyên nhân nan**: là do dữ liệu có độ biến động rất nhỏ giữa các giá trị nên chưa thế mô phỏng theo Hurst được.